<a href="https://colab.research.google.com/github/munkefrugt/StockPrograms/blob/main/Copy_WORKS_macd_banas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 996 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29104 sha256=d68e93f492a5b9f0e23b78bc375f2a65e0d00192907a105721feeabab5be47a3
  Stored in directory: /root/.cache/pip/wheels/31/31/f1/f2ff471bbc5b84a4b973698ceecdd453ae043971791adc3431
Successfully built ta


In [ ]:
import yfinance as yf
import plotly.graph_objects as go

from plotly.subplots import make_subplots

# Install the Python Technical Analysis library
# pip install ta in Qt Console or Powershell prompt
from ta.trend import MACD
from ta.momentum import StochasticOscillator 

In [ ]:
amd_df = yf.download(tickers="AMD", period="1y", interval='1d')
amd_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-24,105.190002,106.050003,104.680000,105.800003,105.800003,26482600
2021-09-27,105.070000,108.440002,103.440002,108.160004,108.160004,51944300
2021-09-28,106.839996,107.650002,101.419998,101.519997,101.519997,74469000
2021-09-29,102.300003,102.660004,99.809998,100.349998,100.349998,52224700
2021-09-30,102.080002,104.440002,101.989998,102.900002,102.900002,57636700
...,...,...,...,...,...,...
2022-09-19,75.389999,77.339996,75.349998,76.769997,76.769997,64966900
2022-09-20,76.080002,76.580002,74.730003,75.250000,75.250000,51129100
2022-09-21,75.440002,78.410004,74.449997,74.480003,74.480003,82069700


In [ ]:
amd_df['MA20'] = amd_df['Adj Close'].ewm(span=20, adjust=False).mean()
amd_df['MA50'] = amd_df['Adj Close'].ewm(span=50, adjust=False).mean()
amd_df

,Open,High,Low,Close,Adj Close,Volume,MA20,MA50
Date,,,,,,,,
2021-09-24,105.190002,106.050003,104.680000,105.800003,105.800003,26482600,105.800003,105.800003
2021-09-27,105.070000,108.440002,103.440002,108.160004,108.160004,51944300,106.024765,105.892552
2021-09-28,106.839996,107.650002,101.419998,101.519997,101.519997,74469000,105.595739,105.721079
2021-09-29,102.300003,102.660004,99.809998,100.349998,100.349998,52224700,105.096145,105.510449
2021-09-30,102.080002,104.440002,101.989998,102.900002,102.900002,57636700,104.886989,105.408078
...,...,...,...,...,...,...,...,...
2022-09-19,75.389999,77.339996,75.349998,76.769997,76.769997,64966900,83.350451,87.592535
2022-09-20,76.080002,76.580002,74.730003,75.250000,75.250000,51129100,82.578980,87.108514
2022-09-21,75.440002,78.410004,74.449997,74.480003,74.480003,82069700,81.807649,86.613278


In [ ]:
fig = go.Figure()

# Define candlestick and moving average lines
candle = go.Candlestick(x=amd_df.index, open=amd_df['Open'],
                       high=amd_df['High'], low=amd_df['Low'],
                       close=amd_df['Close'], name='Candlestick')
ema20 = go.Scatter(x=amd_df.index, y=amd_df['MA20'], 
                   line=dict(color='green', width=1), name="EMA20")

ema50 = go.Scatter(x=amd_df.index, y=amd_df['MA50'], 
                  line=dict(color='orange', width=1), name="EMA50")

# Add plots to the figure
fig.add_trace(candle)
fig.add_trace(ema20)
fig.add_trace(ema50)      

# Add title
fig.update_layout(title="AMD")

# Get rid of empty dates on the weekend
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
                                            
fig.show()

In [ ]:
def plot_macd_stoch(ticker, time_period, time_int):
    # Download stock data
    stock_df = yf.download(tickers=ticker, period=time_period, interval=time_int)
    
    # Define candlestick and moving average lines
    candle = go.Candlestick(x=stock_df.index, open=stock_df['Open'],
                       high=stock_df['High'], low=stock_df['Low'],
                       close=stock_df['Close'])
    # A period of 12 and 26 are commonly used
    stock_df['MA12'] = stock_df['Adj Close'].ewm(span=12, adjust=False).mean()
    stock_df['MA26'] = stock_df['Adj Close'].ewm(span=26, adjust=False).mean()
    
    # Calculate the MACD
    # A signal line uses a period of 9 and when it crosses the other moving
    # average it is a sign to buy or sell
    macd = MACD(close=stock_df['Close'],
             window_slow=26,
             window_fast=12,
             window_sign=9)

    # A Stochastic (stuh ka stuhk) Oscillator is also plotted because it 
    # gives us a signal of an upcoming trend reversal
    # Values range from 0 to 100 and values over 80 are considered to be
    # overbought while those under 20 are considered oversold
    
    # We calculate normally over a 14 day period
    # We are smoothing price data
    sto_os = StochasticOscillator(high=stock_df['High'],
                             close=stock_df['Close'],
                             low=stock_df['Low'],
                             window=14, 
                             smooth_window=3)
    
    # Make 4 subplots on separate rows
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01)

    fig.add_trace(candle, row=1, col=1)

    # Add the histogram, MACD and Signal lines
    
    # The MacD line is a calculation made by subtracting the 26 period
    # EMA from the 12 period EMA
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd(),
                         line=dict(color='blue', width=2)
                        ), row=2, col=1)
    
    # The signal is calculated by taking the average of the last 9 values
    # of the MACD line (The signal line is a slower more smoothed out version of
    # the MACD line)
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd_signal(),
                         line=dict(color='orange', width=2)
                        ), row=2, col=1)
    
    # The histogram shows the difference between the MACD and signal line
    # When the MACD line is below the signal that is a negative value and vice versa
    fig.add_trace(go.Bar(x=stock_df.index, 
                     y=macd.macd_diff()
                    ), row=2, col=1)
    
    # The MacD line is a calculation made by subtracting the 26 period
    # EMA from the 12 period EMA
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd(),
                         line=dict(color='blue', width=2)
                        ), row=2, col=1)
    
    # The signal is calculated by taking the average of the last 9 values
    # of the MACD line (The signal line is a slower more smoothed out version of
    # the MACD line)
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd_signal(),
                         line=dict(color='orange', width=2)
                        ), row=2, col=1)

    # Plot Stochastics
    # This is the faster of the 2 lines called the K line
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=sto_os.stoch(),
                         line=dict(color='blue', width=2)
                        ), row=3, col=1)
    # This line is slower and is known as the D line and it is an average of the K line
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=sto_os.stoch_signal(),
                         line=dict(color='orange', width=2)
                        ), row=3, col=1)
    
    # Draw 20 and 80 lines
    fig.add_hline(y=20, line_width=1, line_dash="dash", line_color="red", row=3, col=1)
    fig.add_hline(y=80, line_width=1, line_dash="dash", line_color="green", row=3, col=1)

    # Add volume 
    fig.add_trace(go.Bar(x=stock_df.index, 
                     y=stock_df['Volume']
                    ), row=4, col=1)

    # Update titles
    fig.update_layout(title=ticker)
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="MACD", row=2, col=1)
    fig.update_yaxes(title_text="StoOs", row=3, col=1)
    fig.update_yaxes(title_text="Volume", row=4, col=1)

    fig.update_layout(height=900, width=1200, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(bounds=["sat", "mon"]), dict(bounds=[16, 9.5], pattern="hour")])

    fig.show()


In [ ]:
plot_macd_stoch("PG", "5d", "5m")

[*********************100%***********************]  1 of 1 completed
